In [4]:
# get the code form github
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 634 (delta 383), reused 564 (delta 313), pack-reused 0
Receiving objects: 100% (634/634), 39.39 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (383/383), done.
Checking out files: 100% (117/117), done.


In [21]:
import pandas as pd
from tqdm import tqdm

In [6]:
df = pd.read_json("/content/Image-Captioning/evaluation_df.json")
df.head()

,file_name,references,hypothesis
0,2431470169_0eeba7d602.jpg,"[[4, 51, 104, 71, 99, 184, 821, 602, 1328, 468...","[4, 64, 62, 104, 34, 4, 264]"
1,2869491449_1041485a6b.jpg,"[[9, 4839, 774, 277, 20, 168, 34, 4, 384], [9,...","[9, 10, 167, 12, 13, 8, 243]"
2,801607443_f15956d1ce.jpg,"[[129, 317, 17, 104, 34, 8, 161, 215, 8, 144, ...","[4, 51, 17, 256, 4, 161]"
3,3716244806_97d5a1fb61.jpg,"[[4, 113, 104, 34, 320, 121, 55, 8, 69], [8, 2...","[4, 164, 19, 112, 104, 34, 4, 161]"
4,2358898017_24496b80e8.jpg,"[[4, 25, 22, 36, 5, 1385, 50, 11, 4, 290, 109]...","[4, 25, 5, 17, 12, 13, 8, 243]"


In [8]:
references = df['references'].to_list()
hypothesis = df['hypothesis'].to_list()

BLEU

In [9]:
from utils import print_scores
print_scores(references, hypothesis)

----- Bleu-n Scores -----
1: 64.08114558472555
2: 46.50725094600124
3: 32.63582641164054
4: 22.48417748427286
-------------------------


(64.08114558472555, 46.50725094600124, 32.63582641164054, 22.48417748427286)

In [100]:
references = []
hypothesis = []
for fname in tqdm(df.file_name.unique()):
  references.append(df[df.file_name==fname].references.to_list()[0])
  hypothesis.append(df[df.file_name==fname].hypothesis.to_list()[0])

100%|██████████| 999/999 [00:01<00:00, 641.34it/s]


In [101]:
len(references), len(hypothesis)

(999, 999)

In [102]:
print_scores(references, hypothesis)

----- Bleu-n Scores -----
1: 64.08114558472555
2: 46.50725094600124
3: 32.63582641164054
4: 22.48417748427286
-------------------------


(64.08114558472555, 46.50725094600124, 32.63582641164054, 22.48417748427286)

### Meteor

In [25]:
# upgrade nltk 
!pip install -U -q nltk

     |████████████████████████████████| 1.5MB 25.4MB/s 


In [26]:
from dataset import build_vocab
DATA_JSON_PATH = 'Image-Captioning/data.json'
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 40000/40000 [00:00<00:00, 245892.79it/s]


In [103]:
preds_tokens = [" ".join([vocab.itos[i] for i in seq]) for seq in tqdm(hypothesis)]

refes_tokens = []
for ref in tqdm(references):
  refes_tokens.append([" ".join([vocab.itos[i] for i in seq]) for seq in ref]) 

100%|██████████| 999/999 [00:00<00:00, 42415.17it/s]


In [30]:
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [31]:
meteor_score(refes_tokens[0], preds_tokens[0])

0.2545362903225807

In [32]:
preds_tokens[0]

'a young boy sitting on a bench'

In [ ]:
from tqdm import tqdm

In [39]:
from statistics import mean, median


meteor_scores = []
for r, h in tqdm(zip(refes_tokens, preds_tokens), total=len(refes_tokens)):
    meteor_scores.append(meteor_score(r, h))

100%|██████████| 999/999 [00:03<00:00, 315.59it/s]


In [41]:
meteor_score?

In [40]:
median(meteor_scores)

0.40760869565217384

In [38]:
mean(meteor_scores)

0.4235476366532275

In [35]:
total_meteor/len(refes_tokens)

0.4235476366532281

### Metrics 

In [53]:
!pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-1t0kabdd
  Running command git clone -q https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-1t0kabdd
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-cp37-none-any.whl size=104312216 sha256=b52d2776ff7c408c22d8dc6376531571d7a210e5d4bec92c3e763b1c74d446d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-5c0oz6a0/wheels/43/c5/1e/8c540096ae74b7789563f1ba2a0e195308ee86f445a0cda110
Successfully built pycocoevalcap


In [115]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.spice.spice import Spice

In [62]:
df.head()

,file_name,references,hypothesis
0,2431470169_0eeba7d602.jpg,"[[4, 51, 104, 71, 99, 184, 821, 602, 1328, 468...","[4, 64, 62, 104, 34, 4, 264]"
1,2869491449_1041485a6b.jpg,"[[9, 4839, 774, 277, 20, 168, 34, 4, 384], [9,...","[9, 10, 167, 12, 13, 8, 243]"
2,801607443_f15956d1ce.jpg,"[[129, 317, 17, 104, 34, 8, 161, 215, 8, 144, ...","[4, 51, 17, 256, 4, 161]"
3,3716244806_97d5a1fb61.jpg,"[[4, 113, 104, 34, 320, 121, 55, 8, 69], [8, 2...","[4, 164, 19, 112, 104, 34, 4, 161]"
4,2358898017_24496b80e8.jpg,"[[4, 25, 22, 36, 5, 1385, 50, 11, 4, 290, 109]...","[4, 25, 5, 17, 12, 13, 8, 243]"


In [105]:
references = {}
hypothesis = {}
for idx, fname in tqdm(enumerate(df.file_name.unique()), total=df.file_name.nunique()):
  hypothesis[idx] = df[df.file_name==fname].hypothesis.to_list()[0]
  references[idx] = df[df.file_name==fname].references.to_list()[0]

  # references.append(df[df.file_name==fname].references.to_list()[0])
  # hypothesis.append(df[df.file_name==fname].hypothesis.to_list()[0])

100%|██████████| 999/999 [00:01<00:00, 649.02it/s]


In [106]:
hypo = {idx: [tokens] for idx, tokens in enumerate(preds_tokens)}
refs = {idx: tokens for idx, tokens in enumerate(refes_tokens)}

In [111]:
len(refs), len(refs)

(999, 999)

In [112]:
hypo[0], refs[0]

(['a young boy sitting on a bench'],
 ['a man sitting at an outside table eating lunch marking on a paper',
  'a single man sitting at a round table reading outside of a shop',
  'man reading a newspaper at an outdoors restaurant',
  'man reading in a chair outside',
  'the man is taking notes at the table'])

In [108]:
score, scores = Bleu(4).compute_score(refs, hypo)
score

{'testlen': 10056, 'reflen': 9831, 'guess': [10056, 9057, 8058, 7059], 'correct': [6444, 3057, 1295, 519]}
ratio: 1.022886786695044


[0.6408114558471916,
 0.46507250945996353,
 0.326358264116369,
 0.22484177484270185]

In [109]:
score, scores = Meteor().compute_score(refs, hypo)
score

0.20691602590745664

In [113]:
score, scores = Rouge().compute_score(refs, hypo)
score

0.47822837472460644

In [114]:
score, scores = Cider().compute_score(refs, hypo)
score

0.5675836611326015

In [116]:
score, scores = Spice().compute_score(refs, hypo)
score

Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.


0.14836015775933178